In [1]:
import torch
import time

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
SIZE = 4
for i in range(0, 5):
    cpu_start = time.time()
    cpu_arr = torch.ones(4 * (10 ** i), 4 * (10 ** i))
    for _ in range(1000):
        cpu_arr += cpu_arr
    print("CPU Time ({}): {}".format(4 * (10 ** i), time.time() - cpu_start))

    gpu_start = time.time()
    gpu_arr = torch.ones(4 * (10 ** i), 4 * (10 ** i)).to(device)
    for _ in range(1000):
        gpu_arr += gpu_arr
    print("GPU Time ({}): {}".format(4 * (10 ** i), time.time() - gpu_start))
    print()

CPU Time (4): 0.003499746322631836
GPU Time (4): 0.00500035285949707

CPU Time (40): 0.0015006065368652344
GPU Time (40): 0.004498958587646484

CPU Time (400): 0.006999492645263672
GPU Time (400): 0.005000591278076172

CPU Time (4000): 4.817999839782715
GPU Time (4000): 0.021498680114746094

